In [1]:
import imageai
import tensorflow
import base64
import cv2
import simplejson as json
import keras
import glob
from imageai.Detection import ObjectDetection
import time
import os
from ftplib import FTP 
import fileinput
start = time.time()

Using TensorFlow backend.


In [2]:
detector = ObjectDetection()

In [3]:
model_path = "./model/rd_model.h5"

In [4]:
detector.setModelTypeAsTinyYOLOv3()

In [5]:
detector.setModelPath(model_path)

In [6]:
detector.loadModel()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
for file_video in glob.glob("./video/*.mp4"):
    video_index = 0
    vidcap = cv2.VideoCapture(str(file_video))
    def getFrame(sec):
        vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        hasFrames,image = vidcap.read()
        if hasFrames:
            cv2.imwrite("./pre_image/image_"+str(video_index)+"_"+str(count)+"_.jpg", image) # save frame as JPG file
        return hasFrames
    sec = 0
    frameRate = 3 #//it will capture image in each 3 second
    count = 1
    success = getFrame(sec)
    while success:
        count += 1
        sec += frameRate
        sec = round(sec, 2)
        success = getFrame(sec)

In [8]:
ftp = FTP()
ftp.connect("10.10.113.43", 21) #21 default port
ftp.login()  #впишите логин если надо
ftp.cwd("OpenCode/ABC")   #задает папку парс на локальном сервере

image_number = 0
for images in glob.glob("./pre_image/*.jpg"):
    count = 0
    output_path = "./output/processed_image_" +str(images.split("_")[2])+ "_" + str(image_number)+".jpg"
    detection = detector.detectObjectsFromImage(input_image=images, output_image_path=output_path)
    for eachItem in detection:
        count +=1
    data = {}
    with open(glob.glob("./output/*.jpg")[image_number], "rb") as imageFile:
        pic = base64.b64encode(imageFile.read())
        data = {'total': count, 'picture' : pic}
    filename = 'data_' + str(image_number) + '.json'
    with open(filename, 'w') as file:
        json.dump(data, file)
    image_number+=1
for j_son in glob.glob("./*.json"):
    fp = open(j_son, 'rb')
    ftp.storbinary('STOR %s' % os.path.basename(j_son), fp, 1024)
    fp.close()

In [9]:
end = time.time()
print('OUT TIME: ', end - start)

OUT TIME:  41.07324528694153
